<a href="https://colab.research.google.com/github/lephuocdat2000/AI-Tempo-Run/blob/main/Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
def Fixed_Coordinate(bbox):
  bbox = np.array(bbox)
  bbox = np.reshape(bbox,(4,2))
  xmin,xmax = np.amin(bbox[:,0],axis = 0),np.amax(bbox[:,0],axis=0)
  ymin,ymax = np.amin(bbox[:,1],axis=0), np.amax(bbox[:,1],axis=0)
  return xmin,ymin,xmax,ymax

def IOU_Calculation(gt_bbox,training_bbox):
  xmin_gt,ymin_gt,xmax_gt,ymax_gt = Fixed_Coordinate(gt_bbox)
  xmin,ymin,xmax,ymax = Fixed_Coordinate(training_bbox)
  if (xmax_gt < xmin) or (ymax_gt<ymin) or (xmin_gt > xmax) or (ymin_gt>ymax): return 0.0
  GT_bbox_area = (xmax_gt - xmin_gt + 1 ) * (ymax_gt - ymin_gt + 1 )
  Pre_bbox_area = (xmax - xmin + 1 ) * (ymax - ymin + 1 )
  xmin = max(xmin_gt,xmin)
  ymin = max(ymin_gt,ymin)
  xmax = min(xmax_gt,xmax)
  ymax = min(ymax_gt,ymax)
  intersection_area = (xmax-xmin+1)*(ymax-ymin+1)
  union_area = GT_bbox_area + Pre_bbox_area - intersection_area
  return intersection_area/union_area

In [7]:
cd /content/gdrive/MyDrive/AITempoRun

/content/gdrive/MyDrive/AITempoRun


In [4]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [11]:
dictionary = os.listdir('result')

for file in dictionary:
    if file.endswith(".txt"):
       f1 = open(os.path.join('result',file),"r").readlines()
       if len(f1)!=0:
          image = cv2.imread(os.path.join('train/images',file[4:-4]+'.jpg'))
          f2 = open(os.path.join('train/labels',file[4:]),"r").readlines()
          f3 = open(os.path.join('Preprocessing Data/test',file[4:]),"w")
          mark = np.zeros((len(f2),))
          for i in range(len(f1)):
              coordinate1 = f1[i].split(",")
              coordinate1 = [int(i) for i in coordinate1]
              IoU_max, selection = 0.0,coordinate1
              for j in range(len(f2)):
                if mark[j]==0:
                    fence = f2[j].rfind(",")
                    coordinate2 = f2[j][0:fence].split(",")
                    coordinate2 = [int(i) for i in coordinate2]
                    IoU = IOU_Calculation(coordinate1,coordinate2)
                    if IoU >= 0.1: 
                        mark[j] = 1
                        if IoU_max<IoU : IoU_max,selection = IoU,coordinate2
              string = ""
              for idx,coor in enumerate(selection):
                    string = string + str(coor) 
                    if idx<len(selection)-1: string+=","
              f3.writelines(string+"\n")      
          # missing = np.where(mark==0)[0]
          # print(file,missing)
          # if len(missing!=0):
          #     for i in missing:
          #        fence = f2[i].rfind(",")
          #        miss_coordinate = f2[i][0:fence]
          #        f3.writelines(miss_coordinate+"\n")   
          f3.close()
          cv2.imwrite(os.path.join('Preprocessing Data/images',file[4:-4]+'.jpg'),image)                       

res_train_1019.txt []
res_train_0271.txt []
res_train_0204.txt []
res_train_0197.txt []
res_train_1167.txt [ 2  5  8 11 12 14 20 21 25 29 30 31 33 34 35 36]
res_train_0120.txt []
res_train_0276.txt []
res_train_0520.txt [13 14 18 19 34 40 45]
res_train_0131.txt [ 0  9 17 19 22 23 36 41]
res_train_0679.txt [ 9 24]
res_train_0542.txt []
res_train_0008.txt [7]
res_train_0417.txt [ 0  3  4  7 11 14 15 23 29 38 49 56]
res_train_0952.txt [0]
res_train_0638.txt [  1  15  17  45 111]
res_train_0492.txt [12 14 25 26 28 33 42 43 44 49]
res_train_1239.txt [32]
res_train_1185.txt []
res_train_0599.txt []
res_train_0742.txt []
res_train_0746.txt []
res_train_1217.txt []
res_train_0949.txt []
res_train_0078.txt []
res_train_1170.txt [ 1  3  7 10 14 19 21 22 28 31 33 34 35 40 43 46 49 50 51 55 57 62]
res_train_0516.txt []
res_train_0909.txt []
res_train_0447.txt [  3   5   8  14  19  26  33  35  38  41  44  47  49  51  52  54  60  61
  65  67  77  82  85  86  92  97  99 100 102 103 104 106 107 108 10

In [ ]:
label_dict = os.listdir('Preprocessing Data/Labeling with CRAFT')
IoU = 0
count = 0
for file in label_dict:
   f1 = open(os.path.join('result',"res_"+file),"r").readlines()
   f2 = open(os.path.join('Preprocessing Data/Labeling with CRAFT',file),"r").readlines()
   if len(f1)!=len(f2): 
     print(file)
     break
   for i in range(len(f1)):
        coordinate1 = f1[i].split(",")
        coordinate1 = [int(i) for i in coordinate1]
        coordinate2 = f2[i].strip().split(",")
        coordinate2 = [int(i) for i in coordinate2]
        IoU += IOU_Calculation(coordinate1,coordinate2)
        count+=1

print(IoU / count)

0.7936060866031666


In [5]:
coordinate1 = [311,358,381,350,388,507,325,516]
coordinate2 = [314,303,355,300,356,343,320,342]
IOU_Calculation(coordinate1,coordinate2)

0.0